[770. Basic Calculator IV](https://leetcode.com/problems/basic-calculator-iv/)

化简式子为polynomial形式

- 没有意识到乘法优先级高于加减法
- 没有意识到-(-)的问题

导致一直按朴素的从左向右处理思想写了，不知道为什么和题目杠上了又添加了`add_parentheses`, `g`这些奇怪的函数，不准备再搞了，复制了一个答案

In [531]:
from collections import Counter
from operator import __add__, __sub__, __mul__

OP = {
    '+': __add__,
    '-': __sub__,
    '*': __mul__
}

# 基于Counter做一个表达式的类
class Exp(Counter):
    def __add__(self, other):
        self.update(other)
        return self

    def __sub__(self, other):
        self.update({k: -v for k, v in other.items()})
        return self

    def __mul__(self, other):
        res = Exp()
        for k1, v1 in self.items():
            for k2, v2 in other.items():
                # print(k1, v1, k2, v2)
                if isinstance(k1, int) and isinstance(k2, int):
                    res.update({k1 * v1 * k2 * v2: 1})
                elif isinstance(k1, int):
                    res.update({k2: k1 * v1 * v2})
                elif isinstance(k2, int):
                    res.update({k1: k2 * v1 * v2})
                else:
                    res.update({tuple(sorted(k1 + k2)): v1 * v2})
        return res
        
    def to_list(self):
        C = 0
        res = []
        for k, v in self.items():
            if v == 0:
                continue
            if isinstance(k, int):
                C += k * v
            else:
                res.append((k, "*".join((str(v),) + k)))
        
        return [j for i, j in sorted(res, key = lambda x: (-len(x[0]), x[0], x[1]))] + ([str(C)] if C else [])
        
class Solution:
    def basicCalculatorIV(self, s: str, evalvars, evalints):
        VARS = {i: j for i, j in zip(evalvars, evalints)}
        # stack[i]: (exp, op)
        stack, num, var = [[Exp(), __add__]], "", ""
        
        def add_parentheses(s):
            res = "#"
            stack = [0]
            flag = [False]
            for i, c in enumerate(s):
                res += c
                if c in '+-*':
                    if c == '*' and res[stack[-1]] in '+-':
                        flag.append(True)
                        res = res[:stack[-1]+2] + '(' + res[stack[-1]+2:]

                    elif c in '+-' and res[stack[-1]] == '*' and flag[-1]:
                        flag.pop()
                        res = res[:-2] + ') ' + c

                    stack[-1] = len(res) - 1
                elif c == '(':
                    flag.append(False)
                    stack.append(0)
                elif c == ')':
                    stack.pop()
                    if flag.pop():
                        res += ')'
                    while flag and flag[-1]:
                        res += ')'
                        flag.pop()

            return res[1:] + (')' if flag[-1] else '')
        def g(s):
            res = ''
            stack = 0
            flag = True
            for i, c in enumerate(s[::-1]):
                if c.isdigit() or c.isalpha():
                    if flag:
                        flag = False
                        stack += 1
                        res += ')'
                else:
                    flag = True
                res += c

            res += '(' * stack

            return res[::-1]

        s = g(add_parentheses(s).replace(")", " )") + " ")

        for i,c in enumerate(s):
            if c == " ":
                exp, op = stack[-1]
                if num:
                    exp = op(exp, Exp([int(num)]))
                
                if var in VARS:
                    exp = op(exp, Exp([VARS[var]]))
                elif var:
                    exp = op(exp, Exp([(var,)]))
                stack[-1] = [exp, op]
                
                num = var = ""
            
            elif c.isdigit():
                num += c
            elif c.isalpha():
                var += c
            elif c in "+-*":
                stack[-1][1] = OP[c]
            elif c == '(':
                stack.append([Exp(), __add__])
            else: # c == ')'
                expa, opa = stack.pop()
                expb, opb = stack[-1]
                stack[-1] = [opb(expb, expa), opb]
            #print(c)
            #print(s[i:])
            #for i in stack:
            #    print(i)
            #print('---------------------------------------------')
        
        return stack[-1][0].to_list()

In [532]:
Solution().basicCalculatorIV("e + 8 - a + 5", ["e"], [1])

['1*a', '14']

In [480]:
Solution().basicCalculatorIV("e - 8 + temperature - pressure", ["e", "temperature"], [1, 12])

['-1*pressure', '5']

In [481]:
Solution().basicCalculatorIV("(e + 8) * (e - 8)", [], [])

['1*e*e', '-64']

In [482]:
Solution().basicCalculatorIV("7 - 7", [], [])

[]

In [483]:
Solution().basicCalculatorIV("((a - b) * (b - c) + (c - a)) * ((a - b) + (b - c) * (c - a))", [], [])

['-1*a*a*b*b',
 '2*a*a*b*c',
 '-1*a*a*c*c',
 '1*a*b*b*b',
 '-1*a*b*b*c',
 '-1*a*b*c*c',
 '1*a*c*c*c',
 '-1*b*b*b*c',
 '2*b*b*c*c',
 '-1*b*c*c*c',
 '2*a*a*b',
 '-2*a*a*c',
 '-2*a*b*b',
 '2*a*c*c',
 '1*b*b*b',
 '-1*b*b*c',
 '1*b*c*c',
 '-1*c*c*c',
 '-1*a*a',
 '1*a*b',
 '1*a*c',
 '-1*b*c']

In [484]:
Solution().basicCalculatorIV("a * b * c + b * a * c * 4", [], [])

['5*a*b*c']

In [485]:
Solution().basicCalculatorIV(
    "5 - (0 - bg * 5) - cb + (aw * 11 + at - cu - 8)",
    ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "aa", "ab", "ac", "ad", "ae", "af", "ag", "ah", "ai", "aj", "ak", "al", "am", "an", "ao", "ap", "aq", "ar", "as", "at", "au", "av", "aw", "ax", "ay", "az", "ba", "bb", "bc", "bd", "be", "bf", "bg", "bh", "bi", "bj", "bk", "bl", "bm", "bn", "bo", "bp", "bq", "br", "bs", "bt", "bu", "bv", "bw", "bx", "by", "bz", "ca", "cb", "cc", "cd", "ce", "cf", "cg", "ch", "ci", "cj", "ck", "cl", "cm", "cn", "co", "cp", "cq", "cr", "cs", "ct", "cu", "cv"],
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 5, 6, 7, 8, 9, 10, 11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 1, 2, 3, 4, 10, 11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
)

['-8']

In [423]:
A = "((bm - bq - 8 - 3) * (by - 0 - 0) + (ae + ch) + 9) - (bq - cv) * (0 - bg * 5) - cv * (n + (q + 2) * (ag + 0 - 4) + 1)"
B = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "aa", "ab", "ac", "ad", "ae", "af", "ag", "ah", "ai", "aj", "ak", "al", "am", "an", "ao", "ap", "aq", "ar", "ast", "at", "au", "av", "aw", "ax", "ay", "az", "ba", "bb", "bc", "bd", "be", "bf", "bg", "bh", "bi", "bj", "bk", "bl", "bm", "bn", "bo", "bp", "bq", "br", "bs", "bt", "bu", "bv", "bw", "bx", "by", "bz", "ca", "cb", "cc", "cd", "ce", "cf", "cg", "ch", "ci", "cj", "ck", "cl", "cm", "cn", "co", "cp", "cq", "cr", "cs", "ct", "cu", "cv"]
C = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 5, 6, 7, 8, 9, 10, 11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 1, 2, 3, 4, 10, 11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [424]:
for I, J in zip(B, C):
    exec(f'{I}={J}')

In [458]:
eval("(bq - cv) * (0 - bg * 5)")

150

In [426]:
eval("((bm - bq - 8 - 3) * (by - 0 - 0) + (ae + ch) + 9)")

11

In [431]:
eval("cv * (n + (q + 2) * (ag + 0 - 4) + 1)")

360

In [459]:
11 - 360 - 150

-499

In [408]:
d = {i: j for i, j in zip(b, c)}

In [ ]:
for 

In [417]:
d['ch']

9

In [475]:
Solution().basicCalculatorIV(A, B, C)

(
((bm - bq - 8 - 3 ) * (by - 0 - 0 ) + (ae + ch ) + 9 ) - ((bq - cv ) * (0 - (bg * 5 ) ) - (cv * (n + ((q + 2 ) * (ag + 0 - 4 ) ) + 1 ) ) 
[Exp(), <built-in function add>]
[Exp(), <built-in function add>]
---------------------------------------------
(
(bm - bq - 8 - 3 ) * (by - 0 - 0 ) + (ae + ch ) + 9 ) - ((bq - cv ) * (0 - (bg * 5 ) ) - (cv * (n + ((q + 2 ) * (ag + 0 - 4 ) ) + 1 ) ) 
[Exp(), <built-in function add>]
[Exp(), <built-in function add>]
[Exp(), <built-in function add>]
---------------------------------------------
b
bm - bq - 8 - 3 ) * (by - 0 - 0 ) + (ae + ch ) + 9 ) - ((bq - cv ) * (0 - (bg * 5 ) ) - (cv * (n + ((q + 2 ) * (ag + 0 - 4 ) ) + 1 ) ) 
[Exp(), <built-in function add>]
[Exp(), <built-in function add>]
[Exp(), <built-in function add>]
---------------------------------------------
m
m - bq - 8 - 3 ) * (by - 0 - 0 ) + (ae + ch ) + 9 ) - ((bq - cv ) * (0 - (bg * 5 ) ) - (cv * (n + ((q + 2 ) * (ag + 0 - 4 ) ) + 1 ) ) 
[Exp(), <built-in function add>]
[Exp(), <bu

['-210']

In [461]:
Exp({0: 1, 330: 1, -120: 1, 220: 1, -80: 1, 10: 1}).to_list()

['360']

In [457]:
Exp({0: 2, 150: 1, 330: -1, -120: -1, 220: -1, -80: -1, 10: -1}).to_list()

['-210']

In [403]:
150 - 330 + 120 - 220 +80 -10

-210

In [397]:
"cv * (n + (q + 2) * (ag + 0 - 4) + 1)"

'cv * (n + (q + 2) * (ag + 0 - 4) + 1)'

In [398]:
f("cv * (n + (q + 2) * (ag + 0 - 4) + 1)")

'cv * (n + ((q + 2) * (ag + 0 - 4)) + 1)'

In [390]:
f("(bk * 8 + 6 * cg * r)")

'(bk * 8 + (6 * cg * r))'

In [391]:
f("5 - (0 - bg * 5) - cb + (aw * 11 + at - cu - 8)")

'5 - (0 - (bg * 5)) - cb + (aw * 11 + at - cu - 8)'

In [392]:
f("a * b * c + b * a * c * 4")

'a * b * c + (b * a * c * 4)'

In [393]:
f("((a - b) * (b - c) + (c - a)) * ((a - b) + (b - c) * (c - a))")

'((a - b) * (b - c) + (c - a)) * ((a - b) + ((b - c) * (c - a)))'

In [345]:
f("((a - b) * (b - c) + (c - a)) * ((a - b) + (b - c) * (c - a))")

'((a - b) * (b - c) + (c - a)) * ((a - b) + ((b - c) * (c - a))'

In [394]:
f("e + 8 - a + 5")

'e + 8 - a + 5'

In [310]:
a = ["2*b*b*c*c","2*a*a*b*c","1*a*c*c*c","1*a*b*b*b","-1*b*c*c*c","-1*b*b*b*c","-1*a*b*c*c","-1*a*b*b*c","-1*a*a*c*c","-1*a*a*b*b","2*a*c*c","2*a*a*b","1*b*c*c","1*b*b*b","-2*a*b*b","-2*a*a*c","-1*c*c*c","-1*b*b*c","1*a*c","1*a*b","-1*b*c","-1*a*a"]

In [311]:
b = ["-1*a*a*b*b","2*a*a*b*c","-1*a*a*c*c","1*a*b*b*b","-1*a*b*b*c","-1*a*b*c*c","1*a*c*c*c","-1*b*b*b*c","2*b*b*c*c","-1*b*c*c*c","2*a*a*b","-2*a*a*c","-2*a*b*b","2*a*c*c","1*b*b*b","-1*b*b*c","1*b*c*c","-1*c*c*c","-1*a*a","1*a*b","1*a*c","-1*b*c"]

In [312]:
a == b

False

In [313]:
sorted(a) == sorted(b)

True

In [488]:
def f(s):
            res = "#"
            stack = [0]
            flag = [False]
            for i, c in enumerate(s):
                res += c
                if c in '+-*':
                    if c == '*' and res[stack[-1]] in '+-':
                        flag.append(True)
                        res = res[:stack[-1]+2] + '(' + res[stack[-1]+2:]

                    elif c in '+-' and res[stack[-1]] == '*' and flag[-1]:
                        flag.popf()
                        res = res[:-2] + ') ' + c

                    stack[-1] = len(res) - 1
                elif c == '(':
                    flag.append(False)
                    stack.append(0)
                elif c == ')':
                    stack.pop()
                    if flag.pop():
                        res += ')'
                    while flag and flag[-1]:
                        res += ')'
                        flag.pop()

            return res[1:] + (')' if flag[-1] else '')

In [529]:
def g(s):
    res = ''
    stack = 0
    flag = True
    for i, c in enumerate(s[::-1]):
        if c.isdigit() or c.isalpha():
            if flag:
                flag = False
                stack += 1
                res += ')'
        else:
            flag = True
        res += c
    
    res += '(' * stack
    
    return res[::-1]

In [530]:
g('ett + 8 - a + 5')

'((((ett) + 8) - a) + 5)'

In [533]:
g('e + 8 - a + 5')

'((((e) + 8) - a) + 5)'

In [534]:
f('e + 8 - a + 5')

'e + 8 - a + 5'